# Google Calendar and people API
Calendar part based on https://medium.com/@ArchTaqi/google-calendar-api-in-your-application-without-oauth-consent-screen-4fcc1f8eb380

# Installation

In [29]:
!pip install google-api-python-client
!pip install psycopg2-binary

     |████████████████████████████████| 3.0 MB 5.5 MB/s 


## Setup

### Set up Google stuff

* enable site wide delegation https://admin.google.com/ac/owl/domainwidedelegation and enable all the scopes below
* create an app https://console.cloud.google.com/home/dashboard
* create a service user and save their credentials to `service-account.json`

In [1]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import pandas as pd


In [2]:
SCOPES = [
          'https://www.googleapis.com/auth/calendar',
          'https://www.googleapis.com/auth/contacts.readonly',
          'https://www.googleapis.com/auth/userinfo.email',
          'https://www.googleapis.com/auth/directory.readonly',
          'https://www.googleapis.com/auth/contacts.other.readonly',


          ]
SERVICE_ACCOUNT_FILE = 'service-account.json' # You should make it an environment variable
CREDENTIALS = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
SUBJECT = 'harry@outlandishideas.co.uk'

# People/contacts

In [3]:
def get_people_service(subject):
  delegated_credentials = CREDENTIALS.with_subject(subject)
  people_service =  build('people', 'v1', credentials=delegated_credentials) 
  return people_service

In [4]:
def get_all_directory(subject):
  people = get_people_service(subject).people()
  #https://googleapis.github.io/google-api-python-client/docs/dyn/people_v1.people.html
  all_people = []
  request = people.listDirectoryPeople(readMask='names,emailAddresses', sources=['DIRECTORY_SOURCE_TYPE_DOMAIN_CONTACT', 'DIRECTORY_SOURCE_TYPE_DOMAIN_PROFILE'])
  response = request.execute()
  while response is not None:
    all_people.extend(response['people'])
    request = people.listDirectoryPeople_next(request, response)
    if not request:
      break
    response = request.execute()
  return all_people

all_directory = get_all_directory(SUBJECT)
len(all_directory)  


32

In [5]:
pd.DataFrame.from_records(pd.json_normalize(all_directory, record_path=['emailAddresses'], meta=['resourceName', 'etag']))

,value,metadata.primary,metadata.verified,metadata.source.type,metadata.source.id,metadata.sourcePrimary,type,formattedType,resourceName,etag
0,ant@outlandishideas.co.uk,True,True,DOMAIN_PROFILE,116681969587251687791,True,NaN,NaN,people/116681969587251687791,%EgcBAgkuNz0+GgMBBwg=
1,ant@outlandish.com,NaN,True,DOMAIN_PROFILE,116681969587251687791,NaN,NaN,NaN,people/116681969587251687791,%EgcBAgkuNz0+GgMBBwg=
2,harry@outlandishideas.co.uk,True,True,DOMAIN_PROFILE,111615362159996029011,True,NaN,NaN,people/111615362159996029011,%EgcBAgkuNz0+GgMBBwg=
3,harry@outlandish.com,NaN,True,DOMAIN_PROFILE,111615362159996029011,NaN,NaN,NaN,people/111615362159996029011,%EgcBAgkuNz0+GgMBBwg=
4,login@outlandishideas.co.uk,True,True,DOMAIN_PROFILE,102365591721057997115,True,NaN,NaN,people/102365591721057997115,%EgcBAgkuNz0+GgMBBwg=
...,...,...,...,...,...,...,...,...,...,...
67,koyan@outlandish.com,NaN,True,DOMAIN_PROFILE,107766348220368688776,NaN,NaN,NaN,people/107766348220368688776,%EgcBAgkuNz0+GgMBBwg=
68,aydarus@outlandishideas.co.uk,True,True,DOMAIN_PROFILE,105180486244090489201,True,NaN,NaN,people/105180486244090489201,%EgcBAgkuNz0+GgMBBwg=
69,aydarus@outlandish.com,NaN,True,DOMAIN_PROFILE,105180486244090489201,NaN,NaN,NaN,people/105180486244090489201,%EgcBAgkuNz0+GgMBBwg=
70,jannik@outlandishideas.co.uk,True,True,DOMAIN_PROFILE,108373280952820605674,True,NaN,NaN,people/108373280952820605674,%EgcBAgkuNz0+GgMBBwg=


In [6]:
def get_all_contacts(subject):
  all_contacts = []
  #https://googleapis.github.io/google-api-python-client/docs/dyn/people_v1.otherContacts.html
  other_contacts = get_people_service(subject).otherContacts()
  request = other_contacts.list(readMask='names,emailAddresses')
  response = request.execute()
  while response is not None:
    all_contacts.extend(response['otherContacts'])
    request = other_contacts.list_next(request, response)
    if not request:
      break
    response = request.execute()
  return all_contacts

all_contacts = get_all_contacts(SUBJECT)
len(all_contacts)

3078

In [7]:
all_people = all_contacts + all_directory
all_addresses = pd.json_normalize(all_people, record_path=['emailAddresses'], meta=['etag', 'resourceName'])
all_addresses

,value,metadata.primary,metadata.source.type,metadata.source.id,metadata.sourcePrimary,type,formattedType,metadata.verified,etag,resourceName
0,choccybic@hortmail.com,True,OTHER_CONTACT,4b3f38a88d77598b,True,NaN,NaN,NaN,%EgcBAgkuNz0+GgECIgx3RmJIWi93RWNtND0=,otherContacts/c5422114772956436875
1,jack.bond@parliament.uk,True,OTHER_CONTACT,43572da1087d907c,True,NaN,NaN,NaN,%EgcBAgkuNz0+GgECIgx1K2p5MzFPWGptZz0=,otherContacts/c4852397293170430076
2,masim@vizob.com,True,OTHER_CONTACT,54a5117d897fa343,True,NaN,NaN,NaN,%EgcBAgkuNz0+GgECIgxrZWU5Mkh5Mjk2ST0=,otherContacts/c6099300501218632515
3,awgandrews@gmail.com,True,OTHER_CONTACT,6c6689f88eccb771,True,NaN,NaN,NaN,%EgcBAgkuNz0+GgECIgxabDAzU0lTUVA0Zz0=,otherContacts/c7811082304361379697
4,aledward@camdennewjournal.com,True,OTHER_CONTACT,4b120fd78af5e0fc,True,NaN,NaN,NaN,%EgcBAgkuNz0+GgECIgxQeFBlZFV5QzJoOD0=,otherContacts/c5409403520849142012
...,...,...,...,...,...,...,...,...,...,...
3221,koyan@outlandish.com,NaN,DOMAIN_PROFILE,107766348220368688776,NaN,NaN,NaN,True,%EgcBAgkuNz0+GgMBBwg=,people/107766348220368688776
3222,aydarus@outlandishideas.co.uk,True,DOMAIN_PROFILE,105180486244090489201,True,NaN,NaN,True,%EgcBAgkuNz0+GgMBBwg=,people/105180486244090489201
3223,aydarus@outlandish.com,NaN,DOMAIN_PROFILE,105180486244090489201,NaN,NaN,NaN,True,%EgcBAgkuNz0+GgMBBwg=,people/105180486244090489201
3224,jannik@outlandishideas.co.uk,True,DOMAIN_PROFILE,108373280952820605674,True,NaN,NaN,True,%EgcBAgkuNz0+GgMBBwg=,people/108373280952820605674


# Calendar events

In [8]:
def get_calendar_service(subject):
  delegated_credentials = CREDENTIALS.with_subject(subject)
  calendar_service =  build('calendar', 'v3', credentials=delegated_credentials) 
  return calendar_service

## Get A User Calendar Events

In [9]:
#https://developers.google.com/calendar/api/v3/reference/events/list

def get_events(subject_email, days_past =0, days_ahead =90):
  start = (datetime.utcnow() - timedelta(days=days_past)).isoformat() + 'Z' # 'Z' indicates UTC time
  end = (datetime.utcnow() + timedelta(days=days_ahead)).isoformat() + 'Z'
  all_events = []
  events = get_calendar_service(subject_email).events()
  request = events.list(calendarId=subject_email,timeMin=start,timeMax=end,orderBy='startTime',singleEvents=True)
  response = request.execute()
  while response is not None:
    all_events.extend(response['items'])
    request = events.list_next(request, response)
    if not request:
      break
    response = request.execute()
  return all_events

all_events = get_events(SUBJECT, days_past = 10, days_ahead = 50)
len(all_events)  

123

In [10]:
events = pd.DataFrame.from_records(all_events)
events

,kind,etag,id,status,htmlLink,created,updated,summary,creator,organizer,start,end,transparency,iCalUID,sequence,attendees,reminders,eventType,location,recurringEventId,originalStartTime,description,guestsCanModify,attachments
0,calendar#event,"""3272605166770000""",2aamcdpkuep0tukep0ntavln2v,confirmed,https://www.google.com/calendar/event?eid=MmFh...,2021-10-27T12:57:43.000Z,2021-11-07T16:29:43.385Z,Feed Cathy's cat,"{'email': 'harry@outlandishideas.co.uk', 'self...","{'email': 'harry@outlandishideas.co.uk', 'self...",{'date': '2021-11-08'},{'date': '2021-11-13'},transparent,2aamcdpkuep0tukep0ntavln2v@google.com,0,"[{'email': 'harry@outlandishideas.co.uk', 'org...","{'useDefault': False, 'overrides': [{'method':...",default,NaN,NaN,NaN,NaN,NaN,NaN
1,calendar#event,"""3270823555962000""",4diirmqma2qlm1uttqv3m441fs_20211108T100500Z,confirmed,https://www.google.com/calendar/event?eid=NGRp...,2021-10-21T16:23:51.000Z,2021-10-28T09:02:57.981Z,Data Orchard stand up,{'email': 'kayleigh@outlandishideas.co.uk'},{'email': 'kayleigh@outlandishideas.co.uk'},"{'dateTime': '2021-11-08T10:05:00Z', 'timeZone...","{'dateTime': '2021-11-08T10:25:00Z', 'timeZone...",NaN,4diirmqma2qlm1uttqv3m441fs@google.com,0,"[{'email': 'harry@outlandishideas.co.uk', 'sel...",{'useDefault': True},default,https://www.whereby.com/quickcomms,4diirmqma2qlm1uttqv3m441fs,"{'dateTime': '2021-11-08T10:05:00Z', 'timeZone...",NaN,NaN,NaN
2,calendar#event,"""3261700724464000""",5m66u524r16fm87pqkb36p8pom_20211108T175000Z,confirmed,https://www.google.com/calendar/event?eid=NW02...,2020-04-15T18:47:53.000Z,2021-09-05T13:59:22.232Z,Toggl Woggl Wobbl!,{'email': 'polly@outlandishideas.co.uk'},{'email': 'polly@outlandishideas.co.uk'},"{'dateTime': '2021-11-08T17:50:00Z', 'timeZone...","{'dateTime': '2021-11-08T18:00:00Z', 'timeZone...",NaN,5m66u524r16fm87pqkb36p8pom@google.com,0,"[{'email': 'harry@outlandishideas.co.uk', 'sel...",{'useDefault': True},default,NaN,5m66u524r16fm87pqkb36p8pom,"{'dateTime': '2021-11-08T17:50:00Z', 'timeZone...",NaN,NaN,NaN
3,calendar#event,"""3270823555962000""",4diirmqma2qlm1uttqv3m441fs_20211109T100500Z,confirmed,https://www.google.com/calendar/event?eid=NGRp...,2021-10-21T16:23:51.000Z,2021-10-28T09:02:57.981Z,Data Orchard stand up,{'email': 'kayleigh@outlandishideas.co.uk'},{'email': 'kayleigh@outlandishideas.co.uk'},"{'dateTime': '2021-11-09T10:05:00Z', 'timeZone...","{'dateTime': '2021-11-09T10:25:00Z', 'timeZone...",NaN,4diirmqma2qlm1uttqv3m441fs@google.com,0,"[{'email': 'harry@outlandishideas.co.uk', 'sel...",{'useDefault': True},default,https://www.whereby.com/quickcomms,4diirmqma2qlm1uttqv3m441fs,"{'dateTime': '2021-11-09T10:05:00Z', 'timeZone...",NaN,NaN,NaN
4,calendar#event,"""3272898334660000""",3730s1sce5q33aev7iibbaqvub,confirmed,https://www.google.com/calendar/event?eid=Mzcz...,2021-11-08T17:32:07.000Z,2021-11-09T09:12:47.330Z,Prospect stand up,{'email': 'kayleigh@outlandishideas.co.uk'},{'email': 'kayleigh@outlandishideas.co.uk'},"{'dateTime': '2021-11-09T10:30:00Z', 'timeZone...","{'dateTime': '2021-11-09T10:50:00Z', 'timeZone...",NaN,3730s1sce5q33aev7iibbaqvub@google.com,1,"[{'email': 'harry@outlandishideas.co.uk', 'sel...",{'useDefault': True},default,https://whereby.com/outlandish-prospect,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,calendar#event,"""3271711566086000""",m9k4ptjig395st47qgb1333ssn_20220105T123000Z,confirmed,https://www.google.com/calendar/event?eid=bTlr...,2017-03-14T17:43:06.000Z,2021-11-02T12:23:03.043Z,BizDev & Comms Circle meeting,{'email': 'polly@outlandishideas.co.uk'},{'email': 'polly@outlandishideas.co.uk'},"{'dateTime': '2022-01-05T12:30:00Z', 'timeZone...","{'dateTime': '2022-01-05T13:15:00Z', 'timeZone...",NaN,m9k4ptjig395st47qgb1333ssn@google.com,6,"[{'email': 'richard@outlandishideas.co.uk', 'r...",{'useDefault': True},default,"https://whereby.com/outlandish-meeting-room, O...",m9k4ptjig395st47qgb1333ssn,"{'dateTime': '20

In [11]:
all_addresses.query('`metadata.source.type` == "DOMAIN_PROFILE"')

,value,metadata.primary,metadata.source.type,metadata.source.id,metadata.sourcePrimary,type,formattedType,metadata.verified,etag,resourceName
3154,ant@outlandishideas.co.uk,True,DOMAIN_PROFILE,116681969587251687791,True,NaN,NaN,True,%EgcBAgkuNz0+GgMBBwg=,people/116681969587251687791
3155,ant@outlandish.com,NaN,DOMAIN_PROFILE,116681969587251687791,NaN,NaN,NaN,True,%EgcBAgkuNz0+GgMBBwg=,people/116681969587251687791
3156,harry@outlandishideas.co.uk,True,DOMAIN_PROFILE,111615362159996029011,True,NaN,NaN,True,%EgcBAgkuNz0+GgMBBwg=,people/111615362159996029011
3157,harry@outlandish.com,NaN,DOMAIN_PROFILE,111615362159996029011,NaN,NaN,NaN,True,%EgcBAgkuNz0+GgMBBwg=,people/111615362159996029011
3158,login@outlandishideas.co.uk,True,DOMAIN_PROFILE,102365591721057997115,True,NaN,NaN,True,%EgcBAgkuNz0+GgMBBwg=,people/102365591721057997115
...,...,...,...,...,...,...,...,...,...,...
3221,koyan@outlandish.com,NaN,DOMAIN_PROFILE,107766348220368688776,NaN,NaN,NaN,True,%EgcBAgkuNz0+GgMBBwg=,people/107766348220368688776
3222,aydarus@outlandishideas.co.uk,True,DOMAIN_PROFILE,105180486244090489201,True,NaN,NaN,True,%EgcBAgkuNz0+GgMBBwg=,people/105180486244090489201
3223,aydarus@outlandish.com,NaN,DOMAIN_PROFILE,105180486244090489201,NaN,NaN,NaN,True,%EgcBAgkuNz0+GgMBBwg=,people/105180486244090489201
3224,jannik@outlandishideas.co.uk,True,DOMAIN_PROFILE,108373280952820605674,True,NaN,NaN,True,%EgcBAgkuNz0+GgMBBwg=,people/108373280952820605674


In [12]:
everyones_events = []
for address in all_addresses.query('`metadata.source.type` == "DOMAIN_PROFILE"').value:
  if "outlandish.com" not in address:
    continue
  print(address)
  everyones_events.extend(get_events(address))

everyones_events

ant@outlandish.com
harry@outlandish.com
login@outlandish.com
toggl@outlandish.com
matt@outlandish.com
pete@outlandish.com
maddy@outlandish.com
thomas@outlandish.com
kayleigh@outlandish.com
ella@outlandish.com
aaron@outlandish.com
ricky@outlandish.com
noel@outlandish.com
nico@outlandish.com
everything@outlandish.com
*@outlandish.com
admin@outlandish.com
schoolcuts@outlandish.com
sangerdata@outlandish.com
abi@outlandish.com
caitlin@outlandish.com
michael@outlandish.com
doug@outlandish.com
mateus@outlandish.com
polly@outlandish.com
lucy@outlandish.com
richard@outlandish.com
jannikko@outlandish.com
droom@outlandish.com
rasmus@outlandish.com
ras@outlandish.com
amil@outlandish.com
koyan@outlandish.com
aydarus@outlandish.com
jannik@outlandish.com


[{'attendees': [{'email': 'harry@outlandish.com',
    'responseStatus': 'needsAction'},
   {'email': 'kayleigh@outlandishideas.co.uk',
    'organizer': True,
    'responseStatus': 'accepted'},
   {'email': 'ant@outlandishideas.co.uk',
    'responseStatus': 'accepted',
    'self': True},
   {'email': 'stuart.hill@prospect.org.uk', 'responseStatus': 'needsAction'},
   {'email': 'koyan@outlandish.com', 'responseStatus': 'accepted'}],
  'created': '2021-11-09T16:19:56.000Z',
  'creator': {'email': 'kayleigh@outlandishideas.co.uk'},
  'description': "Let's have an all-team retro to reflect on what we did this sprint, continue what's working and improve what isn't quite right. \n\nStuart, if this time doesn't work, could you suggest a few other slots? Ideally not on Friday. \n\nThanks",
  'end': {'dateTime': '2021-11-18T14:45:00Z', 'timeZone': 'Europe/London'},
  'etag': '"3273996328948000"',
  'eventType': 'default',
  'htmlLink': 'https://www.google.com/calendar/event?eid=NTFqMGtvbDlxNzNob

In [13]:
pd.DataFrame.from_records(everyones_events)

,kind,etag,id,status,htmlLink,created,updated,summary,description,location,creator,organizer,start,end,iCalUID,sequence,attendees,reminders,eventType,recurringEventId,originalStartTime,visibility,privateCopy,transparency,hangoutLink,conferenceData,guestsCanModify,attachments,guestsCanInviteOthers,source,guestsCanSeeOtherGuests,extendedProperties,colorId
0,calendar#event,"""3273996328948000""",51j0kol9q73housh2dkihmkspr,confirmed,https://www.google.com/calendar/event?eid=NTFq...,2021-11-09T16:19:56.000Z,2021-11-15T17:42:44.474Z,Outlandish x Prospect retro with Stuart,Let's have an all-team retro to reflect on wha...,https://whereby.com/outlandish-prospect,{'email': 'kayleigh@outlandishideas.co.uk'},{'email': 'kayleigh@outlandishideas.co.uk'},"{'dateTime': '2021-11-18T14:00:00Z', 'timeZone...","{'dateTime': '2021-11-18T14:45:00Z', 'timeZone...",51j0kol9q73housh2dkihmkspr@google.com,0,"[{'email': 'harry@outlandish.com', 'responseSt...",{'useDefault': True},default,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,calendar#event,"""3260841887861000""",4tdpi3rkl1g3spsumbfpbpf2qv_20211119T000000Z,confirmed,https://www.google.com/calendar/event?eid=NHRk...,2021-07-13T10:35:33.000Z,2021-10-19T18:49:14.452Z,Non-working day,NaN,NaN,"{'email': 'ant@outlandishideas.co.uk', 'self':...",{'email': 'unknownorganizer@calendar.google.co...,"{'dateTime': '2021-11-19T00:00:00Z', 'timeZone...","{'dateTime': '2021-11-20T00:00:00Z', 'timeZone...",4tdpi3rkl1g3spsumbfpbpf2qv@google.com,0,"[{'email': 'ant@outlandishideas.co.uk', 'displ...",{'useDefault': False},outOfOffice,4tdpi3rkl1g3spsumbfpbpf2qv,"{'dateTime': '2021-11-19T00:00:00Z', 'timeZone...",public,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,calendar#event,"""3260841926190000""",3cmkftfad3aj4t754pg1p28cmf_20211120T000000Z,confirmed,https://www.google.com/calendar/event?eid=M2Nt...,2021-07-13T10:35:49.000Z,2021-10-19T18:49:37.106Z,Non-working day,NaN,NaN,"{'email': 'ant@outlandishideas.co.uk', 'self':...",{'email': 'unknownorganizer@calendar.google.co...,"{'dateTime': '2021-11-20T00:00:00Z', 'timeZone...","{'dateTime': '2021-11-21T00:00:00Z', 'timeZone...",3cmkftfad3aj4t754pg1p28cmf@google.com,0,"[{'email': 'ant@outlandishideas.co.uk', 'displ...",{'useDefault': False},outOfOffice,3cmkftfad3aj4t754pg1p28cmf,"{'dateTime': '2021-11-20T00:00:00Z', 'timeZone...",public,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,calendar#event,"""3260841891813000""",7peta2c98rge1t2pi5anfsfouo_20211121T000000Z,confirmed,https://www.google.com/calendar/event?eid=N3Bl...,2021-07-13T10:35:22.000Z,2021-10-19T18:49:29.574Z,Non-working day,NaN,NaN,"{'email': 'ant@outlandishideas.co.uk', 'self':...",{'email': 'unknownorganizer@calendar.google.co...,"{'dateTime': '2021-11-21T00:00:00Z', 'timeZone...","{'dateTime': '2021-11-22T00:00:00Z', 'timeZone...",7peta2c98rge1t2pi5anfsfouo@google.com,0,"[{'email': 'ant@outlandishideas.co.uk', 'displ...",{'useDefault': False},outOfOffice,7peta2c98rge1t2pi5anfsfouo,"{'dateTime': '2021-11-21T00:00:00Z', 'timeZone...",public,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,calendar#event,"""3260841895743000""",7mj3l0fcea4nm9a47v2smup9up_20211122T000000Z,confirmed,https://www.google.com/calendar/event?eid=N21q...,2021-07-13T10:35:07.000Z,2021-10-19T18:49:48.271Z,Non-working day,NaN,NaN,"{'email': 'ant@outlandishideas.co.uk', 'self':...",{'email': 'unknownorganizer@calendar.google.co...,"{'dateTime': '2021-11-22T00:00:00Z', 'timeZone...","{'dateTime': '2021-11-23T00:00:00Z', 'timeZone...",7mj3l0fcea4nm9a47v2smup9up@google.com,0,"[{'email': 'ant@outlandishideas.co.uk', 'displ...",{'useDefault': False},outOfOffice,7mj3l0fcea4nm9a47v2smup9up,"{'dateTime': '2021-11-22T00:00:00Z', 'timeZone...",public,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729,calendar#event,"""3269983814636000""",6kgajflsbk510aver9ekrl6qcb_20220210T090000Z,confirmed,https://www

## Save the data

In [18]:
from getpass import getpass
from sqlalchemy import create_engine, inspect
from sqlalchemy.dialects.postgresql import JSONB

In [15]:
def get_reporting_db():
  db_host = 'aurora-psql-general.cefrfliziigj.eu-west-1.rds.amazonaws.com'
  db_user = 'a_signalise'
  db_name = 'signalise'
  db_pass = getpass(prompt=f"Database password for user {db_user}@{db_name}")
  engine = create_engine(f"postgresql://{db_user}:{db_pass}@{db_host}/{db_name}")
  db = engine.connect()
  inspector = inspect(engine)
  return db, engine, inspector

db, engine, inspector = get_reporting_db()

Database password for user a_signalise@signalise··········


In [20]:
schema_name = 'gsuite'
db.execute(f"CREATE schema if not exists {schema_name}")
dataframes = dict(people=pd.DataFrame.from_records(all_people), events=pd.DataFrame.from_records(everyones_events))
for table_name, table in dataframes.items():
    type_convert = {}
    table = table.convert_dtypes()
    for column_name, column_type in table.dtypes.items():
        if column_type == 'object':
            type_convert[column_name] = JSONB

        dataframes[table_name] = table

    table.to_sql(name=table_name, con=db, if_exists='replace', dtype=type_convert,  schema=schema_name)

In [24]:
q = "CREATE SCHEMA IF NOT EXISTS reporting"
db.execute(q)

q = """
CREATE OR REPLACE VIEW reporting.events_by_attendee AS (
  select event_id, attendees->>'email', start_time, end_time, summary, description from (
select id as event_id, "start"->>'dateTime' as start_time, "end"->>'dateTime' as end_time, jsonb_array_elements(attendees) as attendees, summary, description from gsuite.events where jsonb_typeof(attendees) = 'array') as event_attendees

)
"""
db.execute(q)

# Editing the calendar

## Create a New Calandar Event

In [ ]:
event = service.events().get(calendarId=SUBJECT,eventId='7e1mpsisedc8l0gqo8leag56hg').execute()

In [ ]:
event

{'created': '2021-11-16T10:57:42.000Z',
 'creator': {'email': 'harry@outlandishideas.co.uk', 'self': True},
 'end': {'date': '2021-11-17'},
 'etag': '"3274120524300000"',
 'eventType': 'default',
 'htmlLink': 'https://www.google.com/calendar/event?eid=N2UxbXBzaXNlZGM4bDBncW84bGVhZzU2aGcgaGFycnlAb3V0bGFuZGlzaGlkZWFzLmNvLnVr',
 'iCalUID': '7e1mpsisedc8l0gqo8leag56hg@google.com',
 'id': '7e1mpsisedc8l0gqo8leag56hg',
 'kind': 'calendar#event',
 'organizer': {'email': 'harry@outlandishideas.co.uk', 'self': True},
 'reminders': {'overrides': [{'method': 'popup', 'minutes': 10},
   {'method': 'email', 'minutes': 430}],
  'useDefault': False},
 'sequence': 0,
 'start': {'date': '2021-11-16'},
 'status': 'confirmed',
 'summary': 'test event',
 'transparency': 'transparent',
 'updated': '2021-11-16T10:57:42.150Z'}

## Delete an Event from a Calandar

In [ ]:
event = service.events().delete(calendarId=SUBJECT,eventId='7e1mpsisedc8l0gqo8leag56hg').execute()

In [ ]:
event

''

## Create an Event into a Calandar

In [ ]:
d = datetime.utcnow().date()
tomorrow = datetime(d.year, d.month, d.day, 10)+timedelta(days=1)
start = tomorrow.isoformat()
end = (tomorrow + timedelta(hours=1)).isoformat()
body={"summary": 'Hello there, Automating calendar', 
      "description": 'Google calendar with python',
      "start": {"dateTime": start, "timeZone": 'Asia/Karachi'}, 
      "end": {"dateTime": end, "timeZone": 'Asia/Karachi'},
     }

In [ ]:
event = service.events().insert(calendarId=SUBJECT,body=body).execute()

In [ ]:
event

{'created': '2021-11-16T11:00:58.000Z',
 'creator': {'email': 'harry@outlandishideas.co.uk', 'self': True},
 'description': 'Google calendar with python',
 'end': {'dateTime': '2021-11-17T06:00:00Z', 'timeZone': 'Asia/Karachi'},
 'etag': '"3274120916978000"',
 'eventType': 'default',
 'htmlLink': 'https://www.google.com/calendar/event?eid=Mm12azlhNzc4c3VyNm1sZnJvdXVjMTcxMWcgaGFycnlAb3V0bGFuZGlzaGlkZWFzLmNvLnVr',
 'iCalUID': '2mvk9a778sur6mlfrouuc1711g@google.com',
 'id': '2mvk9a778sur6mlfrouuc1711g',
 'kind': 'calendar#event',
 'organizer': {'email': 'harry@outlandishideas.co.uk', 'self': True},
 'reminders': {'useDefault': True},
 'sequence': 0,
 'start': {'dateTime': '2021-11-17T05:00:00Z', 'timeZone': 'Asia/Karachi'},
 'status': 'confirmed',
 'summary': 'Hello there, Automating calendar',
 'updated': '2021-11-16T11:00:58.489Z'}